------------------------------------------- 
A* implementaion for PS10 by group no. 250
-------------------------------------------



| S. No. | Name        | ID         | Contribution % |
|--------|-------------|------------|----------------|
| 1      | Rahul       | 2024ad05284| 100            |
| 2      | Brijesh     | 2024ad05270| 100            |
| 3      | Mandan      | 2024ad05331| 100            |
| 4      | Gayathri    | 2024ad05359| 100            |
| 5      | Pranav Deep | 2024ad05376| 100            |

In [1]:
import heapq

# Grid size
rows , cols = 6, 7

# start and goal location
start, goal = (3,0), (3,6)


In [ ]:
# A* Algorithm Implementation Documentation

## Overview
This notebook implements the A* pathfinding algorithm to solve a maze navigation problem. The algorithm finds the optimal path from a start position to a goal position while considering movement costs and heuristic estimates.

## Problem Setup
- **Grid Size**: 6 rows × 7 columns
- **Start Position**: (3, 0) - Row 3, Column 0
- **Goal Position**: (3, 6) - Row 3, Column 6
- **Movement Cost**: Base cost of 3 units per move
- **East Penalty**: Additional cost of 5 units when moving east (right)

## Key Components

### 1. Maze Representation
The maze is represented using two separate 2D arrays:
- `horizontal_walls`: Tracks walls between vertically adjacent cells
- `vertical_walls`: Tracks walls between horizontally adjacent cells

### 2. Cost Function
- **Base Movement Cost**: 3 units for any valid move
- **Eastward Movement Penalty**: Additional 5 units (total 8 units when moving east)
- This creates a preference for paths that minimize eastward movement

### 3. Heuristic Function
- Uses **Manhattan Distance**: |x₁ - x₂| + |y₁ - y₂|
- Weighted by parameter `w` to control algorithm behavior:
  - `w = 1.0`: Optimal A* (guaranteed shortest path)
  - `w > 1.0`: Weighted A* (faster but potentially suboptimal)

## Algorithm Components

In [2]:
# Assume there are no walls

horizontal_walls = [[False for _ in range(cols)] for _ in range(rows)]
vertical_walls = [[False for _ in range(cols)] for _ in range(rows)]


## Wall Configuration

The maze walls are configured using two boolean matrices:

### Horizontal Walls
- `horizontal_walls[i][j] = True` means there's a wall **below** cell (i,j)
- Prevents movement from (i,j) to (i+1,j)

### Vertical Walls  
- `vertical_walls[i][j] = True` means there's a wall **to the right** of cell (i,j)
- Prevents movement from (i,j) to (i,j+1)

### Wall Layout
The current configuration creates a specific maze pattern with strategic wall placements to test the pathfinding algorithm's effectiveness.

In [3]:
# Add Horizontal wall's positions
horizontal_walls[2][0]=True
horizontal_walls[3][1]=True
horizontal_walls[4][1]=True
horizontal_walls[4][2]=True
horizontal_walls[4][3]=True
horizontal_walls[4][4]=True
horizontal_walls[4][5]=True


In [4]:
# Add Vertical wall's positions
vertical_walls[1][0]=True
vertical_walls[2][0]=True
vertical_walls[4][0]=True

vertical_walls[1][1]=True
vertical_walls[2][1]=True
vertical_walls[3][1]=True

vertical_walls[0][2]=True
vertical_walls[1][2]=True
vertical_walls[2][2]=True
vertical_walls[3][2]=True

vertical_walls[1][3]=True
vertical_walls[2][3]=True
vertical_walls[3][3]=True
vertical_walls[4][3]=True

vertical_walls[0][4]=True
vertical_walls[1][4]=True
vertical_walls[2][4]=True
vertical_walls[3][4]=True

vertical_walls[1][5]=True
vertical_walls[2][5]=True
vertical_walls[3][5]=True
vertical_walls[4][5]=True

## Core Algorithm Functions

### Distance and Heuristic Functions
The following functions implement the mathematical foundation of the A* algorithm:

In [5]:
# Manhattan distance
def manhattan_distance(cell1, cell2):
    return abs(cell1[0]-cell2[0]) + abs(cell1[1]-cell2[1])

In [6]:
# Heuristic function
def heuristic_caclc (cell1, cell2, w):
    return w*manhattan_distance(cell1, cell2)

### Movement and Navigation Functions
These functions handle cell connectivity and movement costs:

In [7]:
# Add a neighbor if there is no wall in that direction
def get_neighbors(row, col):
    neighbors = []
    if row > 0 and not horizontal_walls[row - 1][col]:  # check North neighbor cell
        neighbors.append((row - 1, col))
    if row < rows - 1 and not horizontal_walls[row][col]:  # check South neighbor cell
        neighbors.append((row + 1, col))
    if col > 0 and not vertical_walls[row][col - 1]:  # check West neighbor cell
        neighbors.append((row, col - 1))
    if col < cols - 1 and not vertical_walls[row][col]:  # check East neighbor cell
        neighbors.append((row, col + 1))
    return neighbors

In [8]:
# Cost of transition
def transition_cost(current, neighbor):
    default_transition_cost = 3
    if (neighbor[1] > current[1]):
        default_transition_cost+=5 # Additional cost moving east
        
    return default_transition_cost
            

## Main A* Algorithm

The core A* implementation follows these key steps:

### Algorithm Flow:
1. **Initialization**: Start with the initial position in the open list
2. **Main Loop**: Continue until goal is found or open list is empty
3. **Node Selection**: Pop the node with lowest f-cost (f = g + h)
4. **Goal Check**: Return path if current node is the goal
5. **Neighbor Exploration**: Examine all valid neighboring cells
6. **Cost Calculation**: Compute g-cost (actual cost) and f-cost (total estimated cost)
7. **Path Tracking**: Maintain parent relationships for path reconstruction

### Key Data Structures:
- **open_list**: Priority queue of nodes to explore (heapq)
- **comes_from**: Dictionary tracking parent relationships
- **g_cost**: Dictionary storing the actual cost to reach each node

### Cost Function Details:
- **g(n)**: Actual cost from start to node n
- **h(n)**: Heuristic estimate from node n to goal (Manhattan distance × weight)
- **f(n)**: Total estimated cost = g(n) + h(n)

In [9]:
# A* implemntation 
def find_path_with_A_star (start, goal, w =1.0):
    open_list = []
    heapq.heappush(open_list, (0 + heuristic_caclc(start, goal, w), 0, start))

    comes_from = {}
    g_cost = {start: 0}

    while open_list:
        _,current_g,current = heapq.heappop(open_list)

        # Goal is found retrun path and cost
        if current == goal:
            path = []
            while current in comes_from:
                path.append(current)
                current = comes_from[current]
            path.append(start) 
            # Reverse the path to see from start to goal
            path.reverse() 
            return path, g_cost[goal]
            
        # Explore all the neighbors
        for neighbor in get_neighbors(*current): 
            cost = transition_cost(current, neighbor)
            temp_g = current_g + cost
            # Find best possible neighbor to be explored next
            if neighbor not in g_cost or temp_g < g_cost[neighbor]: 
                g_cost[neighbor] = temp_g
                f_cost = temp_g + heuristic_caclc(neighbor, goal, w)
                heapq.heappush(open_list, (f_cost, temp_g, neighbor)) 
                comes_from[neighbor] = current

    return None, None
            

In [10]:
# Print path found and associated cost 
def print_path_cost(start, goal, w):
    path, total_cost = find_path_with_A_star(start, goal, w)
    if path:
        print("Path found:")
        print(" -> ".join(map(str, path)))
        print(f"Total cost: {total_cost}")
    else:
        print("No path found.")    

## Execution and Results

### Testing Scenarios

The implementation is tested with two different weight values to demonstrate the trade-off between optimality and speed:

#### Scenario 1: w = 1.0 (Optimal A*)
- **Behavior**: Guarantees the shortest path
- **Performance**: May explore more nodes
- **Use Case**: When optimality is crucial

#### Scenario 2: w = 2.5 (Weighted A*)
- **Behavior**: Biases toward the goal, potentially finding suboptimal paths
- **Performance**: Generally faster, explores fewer nodes
- **Use Case**: When speed is more important than optimality

### Output Format
For each scenario, the results show:
- **Path**: Sequence of (row, col) coordinates from start to goal
- **Total Cost**: Sum of all movement costs along the path

### Cost Analysis
- Each move has a base cost of 3
- Eastward moves incur an additional penalty of 5 (total cost = 8)
- This penalty encourages the algorithm to find paths that minimize rightward movement

In [11]:
# Find path and cost when w = 1.0
print_path_cost(start, goal, 1.0)

Path found:
(3, 0) -> (4, 0) -> (5, 0) -> (5, 1) -> (5, 2) -> (5, 3) -> (5, 4) -> (5, 5) -> (5, 6) -> (4, 6) -> (3, 6)
Total cost: 60


In [12]:
# Find path and cost when w = 2.5
print_path_cost(start, goal, 2.5)

Path found:
(3, 0) -> (4, 0) -> (5, 0) -> (5, 1) -> (5, 2) -> (5, 3) -> (5, 4) -> (5, 5) -> (5, 6) -> (4, 6) -> (3, 6)
Total cost: 60


In [ ]:
# Technical Details and Analysis

## Algorithm Complexity

### Time Complexity
- **Best Case**: O(b^d) where b is branching factor, d is depth
- **Worst Case**: O(b^d) in the worst case, but typically much better with good heuristics
- **Average Case**: Depends on heuristic quality and maze structure

### Space Complexity
- **O(b^d)** for storing nodes in open and closed lists
- **Additional O(n)** for tracking parent relationships and costs

## Implementation Features

### Optimizations
1. **Priority Queue**: Using `heapq` for efficient node selection
2. **Cost Tracking**: Avoiding re-exploration of expensive paths
3. **Early Termination**: Stops immediately when goal is found

### Design Decisions
1. **Wall Representation**: Separate horizontal and vertical wall arrays for clear directionality
2. **Cost Structure**: Asymmetric costs to model real-world preferences
3. **Weighted Heuristic**: Configurable weight parameter for performance tuning

## Expected Behavior

### Path Characteristics
- The algorithm should prefer paths that minimize eastward movement due to the penalty
- Optimal paths will balance total distance with movement cost penalties
- Different weight values may produce different paths with varying costs

### Performance Considerations
- Higher weights (w > 1.0) trade optimality for speed
- The maze structure significantly impacts the number of nodes explored
- The east penalty creates interesting path dynamics, potentially favoring circuitous routes

## Usage Notes

### Modifying the Configuration
- **Change Start/Goal**: Update the `start` and `goal` variables in cell 3
- **Modify Walls**: Edit the wall configuration in cells 5-7
- **Adjust Costs**: Modify `transition_cost()` function for different movement penalties
- **Test Different Weights**: Change the weight parameter in the execution cells

### Extension Possibilities
- Add diagonal movement capabilities
- Implement bidirectional A* for improved performance
- Add visualization of the maze and solution path
- Compare with other pathfinding algorithms (Dijkstra, BFS)

---

*This implementation demonstrates the core concepts of the A* algorithm while incorporating practical considerations like asymmetric movement costs and configurable heuristic weighting.*